# OPTIMIZING MODEL PARAMETERS
- 모델과 데이터가 준비됐다면 이제 학습할 차례
- 모델을 학습하는 작업은 **반복적인 과정(epoch)** 임
>1. 모델의 예측값 출력
>2. loss function 으로 정답값에 대한 error 계산
>3. error 를 모델의 파라미터로 편미분하여 derivative 구함
>**4. 미분 결과를 이용해 파라미터를 optimize

---
# 1. Prerequisite Code
- **Datasets & DataLoader** 와 **Build Model** 부분을 작성한 뒤에 학습 시작

In [11]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


---
# 2. Hyperparameters
- **하이퍼파라미터란** 모델의 최적화 과정에 관여하는 조절 가능한 파라미터를 뜻함
- 하이퍼파라미터가 달라지면 모델의 학습의 수렴에 영향을 줄 수 있음

>여기선 다음 세 가지 하이퍼파라미터를 다뤄봄
>1. **Number of Epochs** : 데이터셋을 iterate 할 횟수
>2. **Batch size** : 네트워크를 업데이트하기 위해 propagation 에 사용될 데이터 샘플 수
>3. **Learning rate** : 각 batch/epoch 마다 모델의 파라미터를 업데이트 하는 정도 (작으면 학습 과정이 길어지고, 크면 발산할 위험이 있음)

In [6]:
epochs = 5
batch_size =64
learning_rate = 1e-3

---
# 3. Optimization Loop
- 하이퍼파라미터를 설정했다면 이제 **optimization loop** 를 만들 차례
- **학습의 각 iteration(epoch) 을 정의하는 것**

>각 에폭은 두 개의 메인 파트로 이루어짐
>1. **The Train Loop** : 학습 데이터셋에 대한 iteration. **파라미터의 최적화가 이뤄짐**
>2. **The Validation/Test Loop** : 테스트셋에 대한 iteration. **모델의 성능을 확인함**

## 3.1 Loss Function
- 초기의 네트워크는 학습 데이터를 통해 올바른 출력을 하기 어려움
- **Loss Function** 은 모델이 얼마나 출력을 못했는지 측정해줌
- 이 loss function 을 최소화하는 것이 바로 학습임

>- **nn.MSELoss(Mean Square Error)** : regression 작업에 주로 사용하는 loss
>- **nn.NLLLoss(Negative Log Likelihood)** : classification 작업에 주로 사용하는 loss
>- **nn.CrossEntropyLoss** : ```nn.LogSoftmax``` 와 ```nn.NLLLoss``` 를 합친 것

In [7]:
loss_fn = nn.CrossEntropyLoss()

## 3.2 Optimizer
- 최적화란 모델의 파라미터를 loss function 을 줄이는 쪽으로 업데이트하는 작업임
- **Optimization algorithms** 은 이 과정을 어떻게 수행할 것인지 정의함
- 여기선 **SGD(stochastic gradient descent)** 를 사용함
- 파이토치는 **ADAM, RMSProp** 같은 다양한 optimizer 를 제공함

- **optimizer 를 초기화할 땐 최적화할 모델의 파라미터를 등록해야 함**
- learning rate 도 설정해야 함

In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

>**1 학습 루프 당 진행되는 3 단계**
>1. ```optimizer.zero_grad()``` 를 호출하여 파라미터들의 gradient 를 초기화.
>2. ```loss.backward()``` 를 호출하여 에러를 역전파. 이때 각 파라미터에 대한 gradient 를 구함
>3. ```optimizer.step()``` 를 호출하여 파라미터를 업데이트

---
# 4. Full Implementation
- 모델 학습을 위한 ```train_loop```
- 모델 평가를 위한 ```test_loop``` 를 정의

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch%100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f}   [{current:>5d} / {size:>5d}]")
            
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

- loss function 과 optimizer 를 정의했다면 ```train_loop``` 와 ```test_loop``` 에 전달하여 학습 및 평가를 진행
- 성능 향상을 위해 epoch 을 높여볼 수 있음

In [12]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
------------------------------
loss: 2.262627   [    0 / 60000]
loss: 2.262111   [ 6400 / 60000]
loss: 2.245964   [12800 / 60000]
loss: 2.230340   [19200 / 60000]
loss: 2.228112   [25600 / 60000]
loss: 2.198032   [32000 / 60000]
loss: 2.210687   [38400 / 60000]
loss: 2.200469   [44800 / 60000]
loss: 2.182735   [51200 / 60000]
loss: 2.139715   [57600 / 60000]
Test Error: 
 Accuracy: 32.2%, Avg loss: 0.033990 

Epoch 2
------------------------------
loss: 2.196808   [    0 / 60000]
loss: 2.195006   [ 6400 / 60000]
loss: 2.181229   [12800 / 60000]
loss: 2.153560   [19200 / 60000]
loss: 2.139691   [25600 / 60000]
loss: 2.108427   [32000 / 60000]
loss: 2.114838   [38400 / 60000]
loss: 2.113174   [44800 / 60000]
loss: 2.079001   [51200 / 60000]
loss: 2.013290   [57600 / 60000]
Test Error: 
 Accuracy: 40.2%, Avg loss: 0.032367 

Epoch 3
------------------------------
loss: 2.115090   [    0 / 60000]
loss: 2.108305   [ 6400 / 60000]
loss: 2.100135   [12800 / 60000]
loss: 2.053534   [19